# VisBeat Tutorial

Welcome! This notebook will walk you through the basics of using _visbeat_ to manipulate dance in video.

For the **TL;DR** version, check out **VisBeatTLDR.ipynb** and/or **VisBeatTLDR.py**.

- The first part of the tutorial walks you through the basics of using visbeat. 
- The second part demonstrates music-based dance retargeting, and does not require OpenCV. 
- The final part demonstrates dancification using visible impact and visual beats. This requires OpenCV.


<a id='section0'></a>
# Section 0: VisBeat Basics
VisBeat is the public implementation of "__[Visual Rhythm And Beat](http://abedavis.com/visualbeat/)__" [SIGGRAPH 2018]. For an overview of the project, check out our __[Overview Video](https://www.youtube.com/watch?v=K3z68mOLbNo)__ or visit the "__[Project Website](http://abedavis.com/visualbeat/)__".



<a id='section0p1'></a>
## 0.1 -- Importing VisBeat

We will assume you were able to install visbeat using `pip install visbeat`; if that is the case then you should have all of the dependencies necessary for at least the first part of this tutorial.

Import visbeat just like any other python module. In most of my example code I import it as `vb`:<br>
`import visbeat as vb`<br>


<div class="alert alert-block alert-warning">
<b>OpenCV: </b> If you see a warning about OpenCV not being installed, don't worry about it for now. Section 2 of the tutorial requires OpenCV to run visual impact-based warping and visual beat detection, but you can still use retime existing dance video without it (the basic Dancefer() function). We'll come back to this in Section 2.
</div>

***


In [ ]:
# Other imports we will use for convenience in this tutorial
import os, time, matplotlib
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np # Most of the math in visbeat is done with numpy

# We will call visbeat functions through 'vb'
import visbeat as vb

***
<a id='section0p2'></a>
## 0.2 -- Specify a directory for storing files
First things first: we are going to be dealing with a lot of media files, so we need a place to save them.

Specify an asset directory, vb.**VISBEAT_ASSETS_DIR** where videos, audio, and metadata should be stored. All you have to do it provide a path to the root directory -- everything else will be handled by visbeat.

### VISBEAT_ASSETS_DIR organization
See the layout of the assets dir below. There is a 'VideoSources' folder with a separate directory for each video source. Inside each of these directories you will find a 'Versions' folder and a 'Data' folder. The 'Versions' folder stores all variations of the source video: e.g., the original, different resolutions, and any warped outputs we compute. The 'Data' folder saves metadata as well as things like optical flow that are expensive to compute (so we can load results from disk instead of computing them over and over again).

-  VisBeatAssets
    -  VideoSources
        -  Video1
            -  Data
            -  Versions
                -  Original
                -  Warps
        -  Video2
            -  Data
            -  Versions
                -  Original
                -  Warps
        ...

<div class="alert alert-block alert-warning">
<b>Note:</b> It is possible to use visbeat without relying on the assets dir. You can look through my code to see how this is done, but may find that path more difficult. Most complex video editing software needs to manage lots of video versions and metadata on disk and VisBeat is no exception. I should note that VisBeat was extracted from a larger code base I also use in other projects, which is cause for some of the more confusing elements in the code...</div>

***

In [ ]:
# Set the AssetsDir
vb.SetAssetsDir('./VisBeatAssets/');

# For this tutorial we'll also use some files from the test_files dir
test_files_folder = os.path.join('.', 'test_files');

***
<a id='section0p3'></a>
## 0.3 -- Pull some video to play with...

Great! Now lets pull some content to work with. For this we'll use two functions:

vb.**PullVideo(name, source_location)** 
- Pulls a video to the assets directory and puts it in a folder named according to 'name' 
- If source_location is a path to a local file, it will copy that file. 
- if source_location is the url of a Youtube video, it will download that video. Either way, the video is assigned 'name' for future reference.

vb.**LoadVideo(name)** 
- Loads a previously created video according to whatever name you assigned it.

We'll start with a simple synthetic video "./test_files/synth_ball.mp4" which shows a ball bouncing around in a box.

<div class="alert alert-block alert-info">
<b>Tip:</b> Both functions can also take a 'max_height' parameter, which lets you pull different resolutions of the video. For youtube videos, it will request a video with maximum height &lt;= max_height from their servers (what that returns is up to them). For files, you can use max_height to get a lower resolution version of the video with height=max_height at the same aspect ratio.
</div>

<div class="alert alert-block alert-warning">
<b>Warning:</b> Sometimes Youtube's scaling results in weird data... If a video isn't working at a lower resolution (like max_height=240, which is the default), try setting max_height=None. This will get the full res video, which you can then add locally and let visbeat compute lower res versions.
</div>


<!-- <video controls src="synth_ball.mp4">, -->



<!--**NOTE: The default max_height is set to 240 for fast experimentation. If you want to render full resolution, set max_height=None**-->
***

In [ ]:
# Small convenience class to make code more readable and use basic file name if no name is provided;
class SourceMedia:
    def __init__(self, path, name=None, **kwargs):
        self.path = path;
        self._name = name;
        self.__dict__.update(**kwargs)
    @property
    def name(self):         
        if(self._name is not None):
            return self._name;
        else:
            return os.path.splitext(os.path.basename(self.path))[0];

# In this case the name of our SourceMedia will default to 'synth_ball' 
path_to_test_video = os.path.join(test_files_folder, 'synth_ball.mp4');
synth_ball=SourceMedia(path = path_to_test_video);

# Pull the video into VisBeatAssets
vb.PullVideo(name=synth_ball.name, source_location=synth_ball.path);

***
<a id='section0p4'></a>
## 0.4 -- Video, Audio, and Images __[(oh my!)](https://youtu.be/-HrfbV16-FQ?t=16)__

### Video:
You can load previously pulled videos by simply refering to their name with my_video = vb.**LoadVideo** and play them with my_video.**play()**. Loading a video this way also loads some previously evaluated things like optical flow so you don't have to recompute them again. 

### Audio:
Access a video's audio using my_video.**getAudio()**, and play it using my_audio.**play()**

### Images:
Get frame f of a video as a numpy array using my_video.**getFrame(f)**. Or we can get an image object using my_video.**getImageFromFrame(f)**, which we can then display using my_image.**show()**.

<div class="alert alert-block alert-info">
<b>Tip:</b> You can also use the Video, Audio, and Image classes with regular media files (without the rest of visbeat), but you won't get some of the feature saving/loading functionality.
</div>

***


In [ ]:
# Now we can load the video like so...
vid = vb.LoadVideo(name=synth_ball.name)

# And play it in a notebook...
vid.play()

# Get the Audio
aud = vid.getAudio();

# Play the audio
aud.play()

# get an Image object correspoding to a frame number
img = vid.getImageFromFrame(5);

# and show it
img.show()


## 0.5 -- Saving / Loading Features

**NOTE:** This subsection is not critical, but may be useful if you want to experiment with new ideas using visbeat.

Some of the slower video instance methods in visbeat store results as features that are saved to and loaded from disk. This avoids recomputing expensive values every time you run an experiment. You can use this functionality yourself by setting anything pickle-able as a video feature, and calling `save()`. Next time you use `vb.LoadVideo()` to get that video, the feature will load with it from disk.

In [ ]:
# Let's try computing the spectrogram of our video's audio with some really tiny hop size...
# Notice that this calculation takes a bit to perform...

compute_start = time.time()
detailed_spec = vid.audio.getSpectrogram(hop_length=8, force_recompute=True);
compute_time = time.time()-compute_start;

print("Computing took {}s".format(compute_time))

# The resulting array is huge, so lets just take a peek at the first bounce sound...
# images are displayed with row 0 at the top, so it will look upside down here... 
vb.Image.FromGrayScale(gray_data=np.log(1.0+detailed_spec[:,2000:5000]), color_map='jet').show()
plt.title('Computed Segment of Spectrogram')
plt.show()

feature_name = 'name_for_my_spectrogram_feature';

# To avoid computing the spectrogram again, lets save it as a feature of our video.
vid.setFeature(feature_name, detailed_spec);
vid.save(); # saves all features to disk

# Now we'll reload our video from disk, and our feature along with it...
load_start = time.time();
vid_again = vb.LoadVideo(name=synth_ball.name);
reloaded_spec = vid_again.getFeature(feature_name);
load_time = time.time()-load_start;

print("Loading took {}s".format(load_time));

vb.Image.FromGrayScale(gray_data=np.log(1.0+reloaded_spec[:,2000:5000]), color_map='jet').show()
plt.title('Loaded From File (it\'s the same)')
plt.show()

print("Loading was {} times faster...".format(np.true_divide(compute_time, load_time)));


# Section 2: Visible Impact and Visual Beats


## 2.0 Installing OpenCV

Check out the docker container in the git repo for one avenue...


# Below you can see how to dancify the turtle. I will come back to this and add more explanation in Jan when I have more time -Abe

In [ ]:
turtle = vb.PullVideo(name='turtle', source_location='https://www.youtube.com/watch?v=PWD4gktEUAY')
turtle.play()

sexy_science = vb.PullVideo(name='sexy_science', source_location = 'https://www.youtube.com/watch?v=sy-kueG6KlA')

In [ ]:
synch_video_beat = 0;
synch_audio_beat = 0;
nbeats = 16;

output_path = './SexyTurtleScientist.mp4';

warped = vb.Dancify(source_video=turtle, target=sexy_science.getAudio(), synch_video_beat=synch_video_beat,
                    synch_audio_beat=synch_audio_beat, force_recompute=True, warp_type = 'quad',
                    nbeats=nbeats, output_path = output_path)


In [ ]:
warped.play()